In [1]:
# Function to decode operation

In [1]:
%pylab inline 

import mne
from mne.datasets import spm_face
from mne.decoding import GeneralizationAcrossTime
import sys
import os.path as op
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import pandas as pd
from scipy import stats
#Add personal functions to python path
#sys.path.append('/Users/pinheirochagas/Pedro/NeuroSpin/Experiments/Calc_MEG/scripts/decoding/')
sys.path.append('/Volumes/NeuroSpin2T/Calculation_Pedro_2014/scripts/decoding/')
from ecog2mne import ecog2mne
from calc_twoClassClassify import calc_twoClassClassify
from sklearn import svm
from sklearn.cross_validation import cross_val_score, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import StratifiedKFold

/Users/pinheirochagas/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/pinheirochagas/anaconda/lib/python2.7/site-packages/pandas/__init__.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from pandas import hashtable, tslib, lib
/Users/pinheirochagas/anaconda/lib/python2.7/site-packages/pandas/core/common.py:18: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  import pandas.algos as algos
/Users/pinheirochagas/anaconda/lib/python2.7/site-packages/bottleneck/__init__.py:10: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from .reduce import (nansum, nanmean, nanstd, nanvar, nanmin, nanmax,
/Users/pinheirochagas/anaconda/lib/python2.7/site-packages/bottleneck/__init__.py:16: Runti

Populating the interactive namespace from numpy and matplotlib


/Users/pinheirochagas/anaconda/lib/python2.7/site-packages/pandas/io/packers.py:63: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from pandas.msgpack import Unpacker as _Unpacker, Packer as _Packer


In [5]:
#Directories
#data_path = '/Users/pinheirochagas/Pedro/NeuroSpin/Experiments/Calc_MEG/data/mat/'
#result_path = '/Users/pinheirochagas/Pedro/NeuroSpin/Experiments/Calc_MEG/data/decoding/'
data_path = '/Volumes/NeuroSpin2T/Calculation_Pedro_2014/data/mat/'
result_path = '/Volumes/NeuroSpin2T/Calculation_Pedro_2014/data/decoding/'

#Subjects
#subjects = ['s01', 's02', 's03', 's04', 's05', 's06', 's07', 's08', 's09', 's10', 
            #'s11', 's12', 's13', 's14', 's15', 's16', 's17', 's18','s19', 's21', 's22']

    
#subjects = ['s11', 's12', 's13', 's14', 's15', 's16', 's17', 's18','s19', 's21', 's22']

sub = 's01'


#General parameters
baseline = (-0.5, -0.05)
downsmpDec = 4

#Decoding
trainset = 'all'
testset = 'all'
decCond = ['3', '4', '5', '6']

params = {'baseline': baseline, 'downsmpDec': downsmpDec, 
'Classification': decCond, 'trainset': trainset, 'testset': testset}

#Results initialization
all_scores = []
all_diagonals = []

In [12]:
[1 : 106 : 2]

SyntaxError: invalid syntax (<ipython-input-12-8de33c7c3a0b>, line 1)

In [8]:
fname = op.join(data_path, sub + '_ecog.mat') 
epoch = ecog2mne(fname, 'data')


KeyError: u"kind must be one of ['ecg', 'stim', 'eog', 'misc', 'seeg', 'mag', 'eeg', 'grad'], not MEG0113"

In [ ]:

#Baseline-correct & filter data
print('Baseline-correcting data for subject: ' + sub)
epoch.apply_baseline(baseline)
    
#Load condition/behavior info
matfile = sio.loadmat(fname)
print('Loading trialinfo for subject: ' + sub)   
trialinfo = matfile['data']['trialinfo']   

In [3]:

    fname = op.join(data_path, sub + '_ecog.mat') 
    epoch = fldtrp2mne(fname, 'data')

    #Baseline-correct & filter data
    print('Baseline-correcting data for subject: ' + sub)
    epoch.apply_baseline(baseline)
    
    #Load condition/behavior info
    matfile = sio.loadmat(fname)
    print('Loading trialinfo for subject: ' + sub)   
    trialinfo = matfile['data']['trialinfo']   
    
    operator = trialinfo[0][0][0][0][2].T
    operand1 = trialinfo[0][0][0][0][1].T
    operand2 = trialinfo[0][0][0][0][3].T
    absdeviant = trialinfo[0][0][0][0][8].T
    # Merge back and define conditions
    info = pd.DataFrame(data = np.concatenate((operator, operand1, operand2, absdeviant), axis = 1), 
                        columns = ['operation', 'operand1', 'operand2', 'absdeviant'])

    #info['absdeviant'][info['absdeviant'] != 0] = 1
    add_sub = info['operation'] != 0
    
    op0 = info['operand2'] == 0
    op1 = info['operand2'] == 1
    op2 = info['operand2'] == 2
    op3 = info['operand2'] == 3
    
    condA = op0
    condB = op1
    condC = op2
    condD = op3
    
    #Sanity check #1: Plot evoked response
    evoked_condA = epoch[condA].average() # 
    evoked_condB = epoch[condB].average() #   
    evoked_condC = epoch[condC].average() #   
    evoked_condD = epoch[condD].average() #   

    evoked_condA.plot(titles = decCond[0] + ' for subject: ' + sub, show = False)
    evoked_condB.plot(titles = decCond[1] + ' for subject: ' + sub, show = False)
    evoked_condC.plot(titles = decCond[2] + ' for subject: ' + sub, show = False)
    evoked_condD.plot(titles = decCond[3] + ' for subject: ' + sub, show = False)  
    
    plt.show()

    
    #Sanity check #2: Plot GFP
    gfp_condA = np.median(np.std(epoch[condA]._data, axis = 1), axis = 0) #do I need to make sure that the number of trials is similar in the two conditions?
    gfp_condB = np.median(np.std(epoch[condB]._data, axis = 1), axis = 0)
    gfp_condC = np.median(np.std(epoch[condC]._data, axis = 1), axis = 0)
    gfp_condD = np.median(np.std(epoch[condD]._data, axis = 1), axis = 0)

    plt.plot(epoch.times, gfp_condA, color = 'g', label = decCond[0])
    plt.plot(epoch.times, gfp_condB, color = 'r', label = decCond[1])
    plt.plot(epoch.times, gfp_condC, color = 'b', label = decCond[2])
    plt.plot(epoch.times, gfp_condD, color = 'y', label = decCond[3])
    plt.axvline(0, color = 'k') #mark stimulus onset
    plt.legend(loc = 'lower right')
    plt.xlabel('Time (s)')
    plt.ylabel('GFP')
    plt.title('GFP for subject: ' + sub)
    plt.show()
    
    # Decoding
    epochs = epoch[add_sub] #select only target-present trials
    info = info[add_sub]    #info = info[add_sub]
    y = np.array(info['operand2']) #select labels for the conditions
    y = y.astype(numpy.float64)
    # Downsampling for decoding
    epochs.decimate(downsmpDec)

    print('Decoding subject: ' + sub)
    gat, score, diagonal = calc_twoClassClassify(epochs, y, [], [], params)
    gat.plot()
    gat.plot_diagonal()  # plot decoding across time (correspond to GAT diagonal)

    #Store scores of different subjects in the same list
    all_scores.append(score)
    all_diagonals.append(diagonal)
    
#Transform into a numpy array   
all_scores = np.array(all_scores)
all_diagonals = np.array(all_diagonals)

# Save individual results
fname = op.join(result_path, 'Classification_2 ' + decCond[0] + '_vs_' + 
                decCond[1] + '_vs_' + decCond[2] + '_vs_' + decCond[3] + '_Trainset_' + trainset + '_Testset_' + testset) 
np.save(fname, all_scores)

# Compute group averages
group_scores = np.mean(all_scores, 0)
sem_group_scores = stats.sem(all_scores, 0)

group_diagonal = np.mean(all_diagonals, 0)
sem_group_diagonal = stats.sem(all_diagonals, 0)

IndentationError: unexpected indent (<ipython-input-3-73fb96415347>, line 2)

In [ ]:
    fname = op.join(data_path, sub + '_ecog.mat') 
    epoch = fldtrp2mne(fname, 'data')

    #Baseline-correct & filter data
    print('Baseline-correcting data for subject: ' + sub)
    epoch.apply_baseline(baseline)
    
    #Load condition/behavior info
    matfile = sio.loadmat(fname)
    print('Loading trialinfo for subject: ' + sub)   
    trialinfo = matfile['data']['trialinfo']   

In [ ]:
# Plotting  

# Plot GAT
plt.imshow(group_scores, origin = 'lower', extent = [epochs.times[0], epochs.times[len(epochs.times)-1], 
                                                     epochs.times[0], epochs.times[len(epochs.times)-1]]) #flip the matrix around
plt.axvline(0, color = 'k') #mark stimulus onset
plt.axhline(0, color = 'k') #mark stimulus onset
plt.colorbar()
plt.xlabel('Testing Time (s)')
plt.ylabel('Training Time (s)')
plt.title('Group average generalization across time \n Classification: ' + decCond[0] + ' vs ' 
+ decCond[1] + '\n Trainset: ' + trainset + ', Testset: ' + testset)



In [ ]:
# Plot Diagonal
plt.plot(epochs.times, group_diagonal, label = "Classification score")
plt.axvline(0, color = 'k') #mark stimulus onset
plt.axhline(0.5, color = 'k', ls = '--', label = "Chance") #mark chance level
plt.legend(loc = 'upper right')
plt.xlabel('Time (s)')
plt.ylabel('Classification Score (%)')
plt.title('Cl assification: ' + decCond[0] + ' vs ' 
+ decCond[1] + '\n Trainset: ' + trainset + ', Testset: ' + testset)


In [ ]:
all_scores2 = all_scores

In [ ]:
all_scores1 = np.load('/Volumes/NeuroSpin2T/Calculation_Pedro_2014/data/decoding/Classification_ 0_vs_1_vs_2_vs_3_Trainset_all_Testset_all.npy')
all_scores2 = np.load('/Volumes/NeuroSpin2T/Calculation_Pedro_2014/data/decoding/Classification_2 0_vs_1_vs_2_vs_3_Trainset_all_Testset_all.npy')


In [ ]:
all_scores3 = np.concatenate((all_scores1, all_scores2), axis=0)

In [ ]:
size(all_scores3)

In [ ]:
group_scores3 = np.mean(all_scores3, 0)


In [ ]:
# Plot GAT
plt.imshow(group_scores3, origin = 'lower', extent = [epochs.times[0], epochs.times[len(epochs.times)-1], 
                                                     epochs.times[0], epochs.times[len(epochs.times)-1]]) #flip the matrix around
plt.axvline(0, color = 'k') #mark stimulus onset
plt.axhline(0, color = 'k') #mark stimulus onset
plt.colorbar()
plt.xlabel('Testing Time (s)')
plt.ylabel('Training Time (s)')
plt.title('Group average generalization across time \n Classification: ' + decCond[0] + ' vs ' 
+ decCond[1] + '\n Trainset: ' + trainset + ', Testset: ' + testset)



In [ ]:
group_scores3

In [ ]:
# Plot Diagonal
plt.plot(epochs.times, diag3, label = "Classification score")
plt.axvline(0, color = 'k') #mark stimulus onset
plt.axvline(.8, color = 'k') #mark stimulus onset
plt.axvline(1.6, color = 'k') #mark stimulus onset
plt.axvline(2.4, color = 'k') #mark stimulus onset
plt.axvline(3.2, color = 'k') #mark stimulus onset
plt.axvline(3.6, color = 'k') #mark stimulus onset

plt.axhline(0.25, color = 'k', ls = '--', label = "Chance") #mark chance level

plt.legend(loc = 'upper right')
plt.xlabel('Time (s)')
plt.ylabel('Classification Score (%)')
plt.title('Cl assification: ' + decCond[0] + ' vs ' 
+ decCond[1] + '\n Trainset: ' + trainset + ', Testset: ' + testset)


In [ ]:
# Save individual results
fname = op.join(result_path, 'Classification_all ' + decCond[0] + '_vs_' + 
                decCond[1] + '_vs_' + decCond[2] + '_vs_' + decCond[3] + '_Trainset_' + trainset + '_Testset_' + testset) 
np.save(fname, all_scores3)

In [ ]:
scipy.io.savemat('test.mat', dict(x=x, y=y))

In [ ]:
x = np.linspace(0, 2 * np.pi, 100)
y = np.cos(x)

scipy.io.savemat(result_path'test.mat', dict(x=x, y=y))

In [ ]:
fname = op.join(result_path, 'classify_0123.mat')

In [ ]:
fname

In [ ]:
sio.savemat(fname, dict(x=all_scores3))

In [ ]:
np.save('/Volumes/NeuroSpin2T/Calculation_Pedro_2014/data/decoding/epochs_times', epochs.times)




In [ ]:
epochs.times